In [ ]:
import random
import numpy as np
import time
from datetime import timedelta, datetime


class BusStop:
    def __init__(self, passengers_waiting: int):
        self.passengers_waiting = passengers_waiting

    def board_passengers(self, available_seats: int) -> int:
        board_count = min(self.passengers_waiting, available_seats)
        self.passengers_waiting -= board_count
        return board_count

    def add_passengers(self, peak: bool):
        chance = random.random()
        if peak:
            if chance < 0.7:  
                self.passengers_waiting += random.randint(3, 7)
        else:
            if chance < 0.3:  
                self.passengers_waiting += random.randint(1, 3)

class Route:
    def __init__(self, bus_stops_num: int, route_duration: int, peak: bool, upper_limit: int):
        self.num_stops = bus_stops_num
        self.time_between_stops = route_duration // bus_stops_num  
        self.bus_stops = [
            BusStop(random.randint(0, int(upper_limit * (0.7 if peak else 0.3))))
            for _ in range(bus_stops_num)
        ]
class Driver:
    def __init__(self, name: str, shift_duration: int, break_duration: int, schedule_type: str):
        self.name = name
        self.shift_duration = shift_duration
        self.break_duration = break_duration
        self.schedule_type = schedule_type  
        self.time_worked = 0
        self.on_break = False
        self.break_time_remaining = 0

    def work(self, is_peak: bool = False):
        if self.on_break:
            self.break_time_remaining -= 1
            if self.break_time_remaining <= 0:
                self.on_break = False
            return False

        self.time_worked += 1

        if self.schedule_type == "standard":
            if self.time_worked >= self.shift_duration:
                self.start_break()
        elif self.schedule_type == "night":
            if self.time_worked % 120 == 0 and not is_peak: 
                self.start_break()
        return True

    def start_break(self):
        self.on_break = True
        self.break_time_remaining = self.break_duration
        self.time_worked = 0 if self.schedule_type == "standard" else self.time_worked  
class Bus:
    def __init__(self, capacity: int = 20, driver: Driver = None):
        self.capacity = capacity
        self.currently_onboard = 0
        self.driver = driver

    def load_passengers(self, bus_stop):
        if self.driver and not self.driver.work():
            #print(f"Driver {self.driver.name} is on break. Bus cannot load passengers.")
            return 0

        available_seats = self.capacity - self.currently_onboard
        boarding_passengers = bus_stop.board_passengers(available_seats=available_seats)
        self.currently_onboard += boarding_passengers
        return boarding_passengers

    def unload_all_passengers(self):
        exiting_passengers = self.currently_onboard
        self.currently_onboard = 0
        return exiting_passengers
class Simulation:
    def __init__(
        self,
        bus_num: int,
        route_duration: int,
        bus_stops_num: int,
        start_time: np.datetime64,
        end_time: np.datetime64,
        tag: str,
        driver_schedule_type: str,
        driver_shift_duration: int,
        driver_break_duration: int,
        driver_num: int,
    ):
        self.tag = tag
        self.tick = 0
        self.route = Route(bus_stops_num, route_duration, tag in {"monday", "tuesday", "wednesday", "thursday", "friday"}, 100)
        self.time_between_stops = self.route.time_between_stops
        self.end_tick = ((end_time - start_time).astype("timedelta64[m]").item().seconds // self.time_between_stops)
        self.drivers = [
            Driver(
                name=f"Driver {i}",
                shift_duration=driver_shift_duration,
                break_duration=driver_break_duration,
                schedule_type=driver_schedule_type,
            )
            for i in range(driver_num)
        ]
        self.buses = [Bus(driver=self.drivers[i]) for i in range(bus_num)]
        self.bus_positions = [0 for _ in range(bus_num)]
        self.bus_intervals = route_duration // bus_num
        self.start_time = start_time

        self.schedule = {i: [] for i in range(self.route.num_stops)}
        self.bus_start_ticks = [i * self.bus_intervals for i in range(bus_num)]

    def play_simulation(self):
        total_passengers_dropped = 0
        while self.tick < self.end_tick:
            current_time_in_seconds = int((self.start_time + np.timedelta64(self.tick * self.time_between_stops, 'm')).astype("timedelta64[s]").item().seconds % 86400)
            current_hour = current_time_in_seconds // 3600
            is_peak = (7 <= current_hour < 9) or (17 <= current_hour < 19)

            for stop in self.route.bus_stops:
                stop.add_passengers(peak=is_peak if self.tag not in {"saturday", "sunday"} else random.random() < 0.5)

            for i, bus in enumerate(self.buses):
                if self.tick < self.bus_start_ticks[i]:
                    continue

                stop_index = self.bus_positions[i]
                current_time = self.start_time + np.timedelta64(self.tick * self.time_between_stops, 'm')
                self.schedule[stop_index].append(str(current_time))

                if stop_index == len(self.route.bus_stops) - 1:
                    total_passengers_dropped += bus.unload_all_passengers()

                if not bus.driver.work(is_peak):
                    continue

                boarding = bus.load_passengers(self.route.bus_stops[stop_index])
                self.bus_positions[i] = (stop_index + 1) % self.route.num_stops

            self.tick += 1

        remaining_passengers = sum(stop.passengers_waiting for stop in self.route.bus_stops)
        print(f"Simulation finished. Total passengers dropped: {total_passengers_dropped}. Remaining passengers: {remaining_passengers}.")

    def print_schedule(self):
        print("\nBus Schedule:")
        for stop, times in self.schedule.items():
            print(f"Stop {stop}: {', '.join(times)}")

def simulate_with_parameters(bus_num, driver_num, driver_schedule_type, route_duration, bus_stops_num, start_time, end_time, driver_shift_duration, driver_break_duration):
    if driver_num < bus_num: 
        return float("inf") 
    sim = Simulation(
        bus_num=bus_num,
        route_duration=route_duration,
        bus_stops_num=bus_stops_num,
        start_time=start_time,
        end_time=end_time,
        tag="monday",  # Use weekday for peak load
        driver_schedule_type=driver_schedule_type,
        driver_shift_duration=driver_shift_duration,
        driver_break_duration=driver_break_duration,
        driver_num=driver_num
    )
    sim.play_simulation()

    remaining_passengers = sum(stop.passengers_waiting for stop in sim.route.bus_stops)
    return remaining_passengers

def brute_force_optimization(route_duration, bus_stops_num, start_time, end_time, driver_shift_duration, driver_break_duration):
    best_bus_num = None
    best_driver_num = None
    min_remaining_passengers = float("inf")
    start_time_exec = time.time()

    with open("brute_force_results.txt", "w") as f, open("best_results_brute_force.txt", "w") as best_f:
        for bus_num in range(8, 21):
            for driver_num in range(bus_num, 21):  # Ensure drivers >= buses
                for schedule_type in ["standard", "night"]:
                    remaining_passengers = simulate_with_parameters(
                        bus_num=bus_num,
                        driver_num=driver_num,
                        driver_schedule_type=schedule_type,
                        route_duration=route_duration,
                        bus_stops_num=bus_stops_num,
                        start_time=start_time,
                        end_time=end_time,
                        driver_shift_duration=driver_shift_duration,
                        driver_break_duration=driver_break_duration,
                    )

                    if remaining_passengers < min_remaining_passengers:
                        best_bus_num = bus_num
                        best_driver_num = driver_num
                        min_remaining_passengers = remaining_passengers

                    f.write(f"Buses: {bus_num}, Drivers: {driver_num}, Type: {schedule_type}, Remaining: {remaining_passengers}\n")

        exec_time = time.time() - start_time_exec
        best_f.write(f"Best Buses: {best_bus_num}, Best Drivers: {best_driver_num}, Execution Time: {exec_time:.2f} seconds\n")
    return best_bus_num, best_driver_num

def genetic_algorithm_optimization(
    route_duration, 
    bus_stops_num, 
    start_time, 
    end_time, 
    driver_shift_duration, 
    driver_break_duration,
    population_size=10, 
    generations=50, 
    patience=10, 
    log_file="genetic_algorithm_log.txt",
):
    start_time_exec = time.time()

    population = [
        {
            "bus_num": random.randint(bus_stops_num, 20),
            "driver_num": random.randint(bus_stops_num, 20),
            "schedule_type": random.choice(["standard", "night"]),
        }
        for _ in range(population_size)
    ]

    best_solution = None
    best_fitness = float("inf")
    generations_without_improvement = 0

    with open(log_file, "w") as log, open("best_results_genetic_algorithm.txt", "w") as best_f:
        log.write("Generation,Bus_Num,Driver_Num,Schedule_Type,Remaining_Passengers\n")

        for generation in range(generations):
            fitness_scores = []
            for individual in population:
                if individual["driver_num"] < individual["bus_num"]:  # Ensure drivers >= buses
                    fitness_scores.append((individual, float("inf")))
                    continue

                remaining_passengers = simulate_with_parameters(
                    bus_num=individual["bus_num"],
                    driver_num=individual["driver_num"],
                    driver_schedule_type=individual["schedule_type"],
                    route_duration=route_duration,
                    bus_stops_num=bus_stops_num,
                    start_time=start_time,
                    end_time=end_time,
                    driver_shift_duration=driver_shift_duration,
                    driver_break_duration=driver_break_duration,
                )

                fitness_scores.append((individual, remaining_passengers))

                if remaining_passengers < best_fitness:
                    best_solution = individual
                    best_fitness = remaining_passengers
                    generations_without_improvement = 0

            fitness_scores.sort(key=lambda x: x[1])

            top_individuals = [fs[0] for fs in fitness_scores[: population_size // 2]]

            new_population = []
            for _ in range(population_size):
                parent1, parent2 = random.sample(top_individuals, 2)
                child = {
                    "bus_num": max(1, (parent1["bus_num"] + parent2["bus_num"]) // 2),
                    "driver_num": max(parent1["bus_num"], (parent1["driver_num"] + parent2["driver_num"]) // 2),
                    "schedule_type": random.choice([parent1["schedule_type"], parent2["schedule_type"]]),
                }

                if random.random() < 0.1:
                    child["bus_num"] = max(1, min(20, child["bus_num"] + random.choice([-1, 1])))
                    child["driver_num"] = max(child["bus_num"], min(20, child["driver_num"] + random.choice([-1, 1])))

                new_population.append(child)

            population = new_population
            generations_without_improvement += 1

            if generations_without_improvement >= patience:
                break

        exec_time = time.time() - start_time_exec
        best_f.write(f"Best Solution: {best_solution}, Execution Time: {exec_time:.2f} seconds\n")

    return best_solution

route_duration = 70
bus_stops_num = 7
start_time = np.datetime64('2024-12-14T06:00')
end_time = np.datetime64('2024-12-15T03:00')
driver_shift_duration = 420
driver_break_duration = 60

best_bus_num, best_driver_num = brute_force_optimization(
    route_duration=route_duration,
    bus_stops_num=bus_stops_num,
    start_time=start_time,
    end_time=end_time,
    driver_shift_duration=driver_shift_duration,
    driver_break_duration=driver_break_duration,
)
print(f"Best configuration from brute force: Buses = {best_bus_num}, Drivers = {best_driver_num}")

best_solution = genetic_algorithm_optimization(
    route_duration=route_duration,
    bus_stops_num=bus_stops_num,
    start_time=start_time,
    end_time=end_time,
    driver_shift_duration=driver_shift_duration,
    driver_break_duration=driver_break_duration,
)
print(f"Best configuration from genetic algorithm: {best_solution}")

Simulation finished. Total passengers dropped: 58387. Remaining passengers: 2858.
Simulation finished. Total passengers dropped: 51972. Remaining passengers: 8299.
Simulation finished. Total passengers dropped: 58269. Remaining passengers: 2470.
Simulation finished. Total passengers dropped: 52802. Remaining passengers: 8436.
Simulation finished. Total passengers dropped: 58664. Remaining passengers: 2829.
Simulation finished. Total passengers dropped: 53045. Remaining passengers: 7990.
Simulation finished. Total passengers dropped: 58398. Remaining passengers: 2416.
Simulation finished. Total passengers dropped: 53201. Remaining passengers: 8107.
Simulation finished. Total passengers dropped: 58837. Remaining passengers: 2776.
Simulation finished. Total passengers dropped: 53085. Remaining passengers: 8270.
Simulation finished. Total passengers dropped: 58391. Remaining passengers: 2611.
Simulation finished. Total passengers dropped: 53221. Remaining passengers: 8071.
Simulation finis

In [14]:
def simulate_week_with_parameters(bus_num, driver_num, driver_schedule_type, route_duration, bus_stops_num, start_time, end_time, driver_shift_duration, driver_break_duration):
    days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    total_remaining_passengers = 0  # Суммируем оставшихся пассажиров за неделю

    for day in days:
        sim = Simulation(
            bus_num=bus_num,
            route_duration=route_duration,
            bus_stops_num=bus_stops_num,
            start_time=start_time,
            end_time=end_time,
            tag=day,
            driver_schedule_type=driver_schedule_type,
            driver_shift_duration=driver_shift_duration,
            driver_break_duration=driver_break_duration,
            driver_num=driver_num
        )
        sim.play_simulation()
        total_remaining_passengers += sum(stop.passengers_waiting for stop in sim.route.bus_stops)

    return total_remaining_passengers

def brute_force_optimization(route_duration, bus_stops_num, start_time, end_time, driver_shift_duration, driver_break_duration):
    best_bus_num = None
    best_driver_num = None
    best_schedule_type = None
    min_remaining_passengers = float("inf")
    start_time_exec = time.time()

    with open("brute_force_weekly_results.txt", "w") as f, open("best_results_brute_force_weekly.txt", "w") as best_f:
        for bus_num in range(8, 21):
            for driver_num in range(bus_num, 21):  # Ensure drivers >= buses
                for schedule_type in ["standard", "night"]:
                    remaining_passengers = simulate_week_with_parameters(
                        bus_num=bus_num,
                        driver_num=driver_num,
                        driver_schedule_type=schedule_type,
                        route_duration=route_duration,
                        bus_stops_num=bus_stops_num,
                        start_time=start_time,
                        end_time=end_time,
                        driver_shift_duration=driver_shift_duration,
                        driver_break_duration=driver_break_duration,
                    )

                    if remaining_passengers < min_remaining_passengers:
                        best_bus_num = bus_num
                        best_driver_num = driver_num
                        best_schedule_type = schedule_type
                        min_remaining_passengers = remaining_passengers

                    f.write(f"Buses: {bus_num}, Drivers: {driver_num}, Type: {schedule_type}, Remaining: {remaining_passengers}\n")

        exec_time = time.time() - start_time_exec
        best_f.write(f"Best Buses: {best_bus_num}, Best Drivers: {best_driver_num}, Type: {best_schedule_type}, Execution Time: {exec_time:.2f} seconds\n")
    return best_bus_num, best_driver_num, best_schedule_type

def genetic_algorithm_optimization(
    route_duration, 
    bus_stops_num, 
    start_time, 
    end_time, 
    driver_shift_duration, 
    driver_break_duration, 
    population_size=10, 
    generations=50, 
    patience=10, 
    log_file="genetic_algorithm_weekly_log.txt"
):
    start_time_exec = time.time()

    population = [
        {
            "bus_num": random.randint(1, 20),
            "driver_num": random.randint(1, 20),
            "schedule_type": random.choice(["standard", "night"]),
        }
        for _ in range(population_size)
    ]

    best_solution = None
    best_fitness = float("inf")
    generations_without_improvement = 0

    with open(log_file, "w") as log, open("best_results_genetic_algorithm_weekly.txt", "w") as best_f:
        log.write("Generation,Bus_Num,Driver_Num,Schedule_Type,Remaining_Passengers\n")

        for generation in range(generations):
            fitness_scores = []
            for individual in population:
                if individual["driver_num"] < individual["bus_num"]:  # Ensure drivers >= buses
                    fitness_scores.append((individual, float("inf")))
                    continue

                remaining_passengers = simulate_week_with_parameters(
                    bus_num=individual["bus_num"],
                    driver_num=individual["driver_num"],
                    driver_schedule_type=individual["schedule_type"],
                    route_duration=route_duration,
                    bus_stops_num=bus_stops_num,
                    start_time=start_time,
                    end_time=end_time,
                    driver_shift_duration=driver_shift_duration,
                    driver_break_duration=driver_break_duration,
                )

                fitness_scores.append((individual, remaining_passengers))

                if remaining_passengers < best_fitness:
                    best_solution = individual
                    best_fitness = remaining_passengers
                    generations_without_improvement = 0

            fitness_scores.sort(key=lambda x: x[1])

            top_individuals = [fs[0] for fs in fitness_scores[: population_size // 2]]

            new_population = []
            for _ in range(population_size):
                parent1, parent2 = random.sample(top_individuals, 2)
                child = {
                    "bus_num": max(1, (parent1["bus_num"] + parent2["bus_num"]) // 2),
                    "driver_num": max(parent1["bus_num"], (parent1["driver_num"] + parent2["driver_num"]) // 2),
                    "schedule_type": random.choice([parent1["schedule_type"], parent2["schedule_type"]]),
                }

                if random.random() < 0.1:
                    child["bus_num"] = max(1, min(20, child["bus_num"] + random.choice([-1, 1])))
                    child["driver_num"] = max(child["bus_num"], min(20, child["driver_num"] + random.choice([-1, 1])))

                new_population.append(child)

            population = new_population
            generations_without_improvement += 1

            if generations_without_improvement >= patience:
                break

        exec_time = time.time() - start_time_exec
        best_f.write(f"Best Solution: {best_solution}, Execution Time: {exec_time:.2f} seconds\n")

    return best_solution



In [29]:
best_bus_num, best_driver_num, best_schedule_type = brute_force_optimization(
    route_duration=route_duration,
    bus_stops_num=bus_stops_num,
    start_time=start_time,
    end_time=end_time,
    driver_shift_duration=driver_shift_duration,
    driver_break_duration=driver_break_duration,
)

print(f"Лучший результат brute-force:")
print(f"Автобусы: {best_bus_num}, Водители: {best_driver_num}, Тип расписания: {best_schedule_type}")
best_solution = genetic_algorithm_optimization(
    route_duration=route_duration,
    bus_stops_num=bus_stops_num,
    start_time=start_time,
    end_time=end_time,
    driver_shift_duration=driver_shift_duration,
    driver_break_duration=driver_break_duration,
    population_size=10,  
    generations=50,  
    patience=10  
)

print(f"Лучший результат генетического алгоритма:")
print(best_solution)
simulation = Simulation(
    bus_num=best_solution["bus_num"],
    route_duration=route_duration,
    bus_stops_num=bus_stops_num,
    start_time=start_time,
    driver_num=best_solution['driver_num'],
    end_time=end_time,
    tag="monday",  # Пример дня для анализа
    driver_schedule_type=best_solution["schedule_type"],
    driver_shift_duration=driver_shift_duration,
    driver_break_duration=driver_break_duration,
)

simulation.play_simulation()
simulation.print_schedule()
with open("results_summary.txt", "w") as f:
    f.write(f"Brute-force: Buses = {best_bus_num}, Drivers = {best_driver_num}, Schedule Type = {best_schedule_type}\n")
    f.write(f"Genetic Algorithm: {best_solution}\n")


Simulation finished. Total passengers dropped: 58315. Remaining passengers: 2714.
Simulation finished. Total passengers dropped: 58335. Remaining passengers: 2415.
Simulation finished. Total passengers dropped: 58425. Remaining passengers: 2974.
Simulation finished. Total passengers dropped: 58453. Remaining passengers: 2591.
Simulation finished. Total passengers dropped: 58307. Remaining passengers: 2927.
Simulation finished. Total passengers dropped: 109206. Remaining passengers: 291.
Simulation finished. Total passengers dropped: 108650. Remaining passengers: 332.
Simulation finished. Total passengers dropped: 53441. Remaining passengers: 7998.
Simulation finished. Total passengers dropped: 52583. Remaining passengers: 8064.
Simulation finished. Total passengers dropped: 53317. Remaining passengers: 7914.
Simulation finished. Total passengers dropped: 53220. Remaining passengers: 8313.
Simulation finished. Total passengers dropped: 53178. Remaining passengers: 8638.
Simulation finis

In [28]:
import pandas as pd

def generate_schedule_tables(simulation, is_weekday=True):
    day_type = "Weekday" if is_weekday else "Weekend"
    schedule_tables = {}
    
    for stop, times in simulation.schedule.items():
        filtered_times = []
        for time in times:
            time_obj = pd.Timestamp(time)
            hour = time_obj.hour
            if (6 <= hour < 24) or (hour < 3):  # Рабочие часы: с 6 утра до 3 ночи
                filtered_times.append(time_obj.strftime("%H:%M"))
        
        # Удаляем дубликаты и сортируем
        unique_times = sorted(set(filtered_times))

        # Создаем DataFrame для расписания
        df = pd.DataFrame({"Time": unique_times})
        schedule_tables[f"Stop {stop} ({day_type})"] = df

    return schedule_tables


def save_schedule_to_file(schedule_tables, filename_prefix):
    for stop, table in schedule_tables.items():
        filename = f"{filename_prefix}_{stop.replace(' ', '_').replace('(', '').replace(')', '')}.csv"
        table.to_csv(filename, index=False, encoding="utf-8")
        print(f"Saved {stop} schedule to file: {filename}")

def decide(best_bus_num_g, best_driver_num_g, best_schedule_g, best_bus_num_b, best_driver_num_b, best_schedule_b):
    g = best_bus_num_g < best_bus_num_b + best_driver_num_g < best_bus_num_b
    if g>1:
        return best_bus_num_g,best_driver_num_g,best_schedule_g
    else:
        return best_bus_num_b, best_driver_num_b, best_schedule_b
    
bus_num, driver_num, schedule = decide(best_solution['bus_num'],best_solution['driver_num'], best_solution['schedule_type'], best_bus_num,best_driver_num,best_schedule_type)
print(bus_num,driver_num,schedule)
simulation = Simulation(
    bus_num=bus_num,  # Примерное количество автобусов
    route_duration=70,
    bus_stops_num=7,
    start_time=np.datetime64("2024-12-18T06:00"),
    end_time=np.datetime64("2024-12-18T23:00"),
    tag="monday",  # Пример буднего дня
    driver_schedule_type=schedule,
    driver_shift_duration=420,
    driver_break_duration=60,
    driver_num = driver_num
)
simulation.play_simulation()
simulation.print_schedule()
weekday_tables = generate_schedule_tables(simulation, is_weekday=True)
save_schedule_to_file(weekday_tables, filename_prefix="weekday_schedule")


weekend_tables = generate_schedule_tables(simulation, is_weekday=False)
save_schedule_to_file(weekend_tables, filename_prefix="weekend_schedule")


14 16 standard
Simulation finished. Total passengers dropped: 40813. Remaining passengers: 0.

Bus Schedule:
Stop 0: 2024-12-18T06:00, 2024-12-18T06:50, 2024-12-18T07:10, 2024-12-18T07:40, 2024-12-18T08:00, 2024-12-18T08:20, 2024-12-18T08:30, 2024-12-18T08:50, 2024-12-18T09:10, 2024-12-18T09:20, 2024-12-18T09:30, 2024-12-18T09:40, 2024-12-18T10:00, 2024-12-18T10:10, 2024-12-18T10:20, 2024-12-18T10:30, 2024-12-18T10:40, 2024-12-18T10:50, 2024-12-18T11:00, 2024-12-18T11:10, 2024-12-18T11:20, 2024-12-18T11:30, 2024-12-18T11:40, 2024-12-18T11:50, 2024-12-18T11:50, 2024-12-18T12:00, 2024-12-18T12:10, 2024-12-18T12:20, 2024-12-18T12:30, 2024-12-18T12:40, 2024-12-18T12:40, 2024-12-18T12:50, 2024-12-18T13:00, 2024-12-18T13:00, 2024-12-18T13:10, 2024-12-18T13:20, 2024-12-18T13:30, 2024-12-18T13:30, 2024-12-18T13:40, 2024-12-18T13:50, 2024-12-18T13:50, 2024-12-18T14:00, 2024-12-18T14:10, 2024-12-18T14:10, 2024-12-18T14:20, 2024-12-18T14:20, 2024-12-18T14:30, 2024-12-18T14:40, 2024-12-18T14:40, 2